In [184]:
import pandas as pd
import json
from geopy.distance import distance

import pymongo
from pymongo import MongoClient

import src.extract as extract

In [171]:
with open('parameters.json') as f:
    params = json.load(f)

params

{'ftp_days': 4,
 'gtfs_period': 0,
 'database': 'muni_prediction_data',
 'avl_collection': 'avl_raw',
 'labeled_collection': 'labeled_trips',
 'chunk_collection': 'chunk_details',
 'bus': '33',
 'direction': 0,
 'chunks': [2, 6]}

In [172]:
gtfs_lookup = pd.read_csv('data/gtfs_lookup.csv')

In [197]:
client = MongoClient('localhost', 27017)

db = client['muni_prediction_data']
raw_coll = db['avl_raw']
trip_coll = db['labeled_trips']
chunk_coll = db['chunk_details']

In [198]:
chunk_coll.find_one()

{'_id': ObjectId('5aef9bdd3ad39e9acce2689a'),
 'number_chunks': 2,
 'chunks': {'1': {'chunk_stop_id': 3329,
   'chunk_sum_dist': 5837,
   'chunk_stop_seq': 24,
   'chunk_stop_lat': 37.760702,
   'chunk_stop_lon': -122.437294,
   'chunk_stop_name': '18th St & Diamond St',
   'chunk_dist': 5837},
  '2': {'chunk_stop_id': 3511,
   'chunk_sum_dist': 10494,
   'chunk_stop_seq': 45,
   'chunk_stop_lat': 37.751407,
   'chunk_stop_lon': -122.40668,
   'chunk_stop_name': '25th St & Potrero Ave',
   'chunk_dist': 4657}}}

In [199]:
trip_coll.find_one()

{'_id': ObjectId('5aef45623ad39e87335999fa'),
 'HEADING': '69.0',
 'LATITUDE': '37.78698',
 'LONGITUDE': '-122.45644',
 'PREDICTABLE': '1',
 'REPORT_TIME': '11/23/2016 00:27:33',
 'REV': '1526',
 'SPEED': '0.833',
 'TRAIN_ASSIGNMENT': '3305',
 'VEHICLE_TAG': '5405',
 'minutes_noon_sqr': 480249,
 'sched_time_diff_seconds': 147,
 'service_id': 1,
 'time_stamp': 1479889653.0,
 'trip_id': 7253717,
 'trip_id_iso': '7253717_2016-11-23_ENX2Y',
 'trip_start': 1,
 'chunk_2': '1',
 'chunk_6': '1'}

In [181]:
all_trips = trip_coll.distinct('trip_id_iso')

In [193]:
for trip in all_trips:
    
    for chunk in chunk_coll.find():

        chnk_num = "chunk_" + str(chunk['number_chunks'])
        
    #     start_ts = 0

        for seq, chunk_info in chunk['chunks'].items():

            best_dist = {
                'stop_dist':100000, 
                'time_stamp': 0}

            filter_search = {
                "trip_id_iso": trip,
                chnk_num: {"$exists": False}
            }
            
            for doc in trip_coll.find(filter_search).sort('time_stamp'):

                doc_lat = doc['LATITUDE']
                doc_lon = doc['LONGITUDE']

                cnk_lat = chunk_info['chunk_stop_lat']
                cnk_lon = chunk_info['chunk_stop_lon']

                doc_dist = distance((doc_lat, doc_lon), (cnk_lat, cnk_lon)).m

                if doc_dist < best_dist['stop_dist']:
                    best_dist['stop_dist'] = doc_dist
                    best_dist['time_stamp'] = doc['time_stamp']
                    
            print (best_dist)

#             label_search = {
#                 "trip_id_iso": trip,
#                 "time_stamp": {
#                     "$lt": best_dist['time_stamp']
#                 }
#             }

#             for doc in trip_coll.find(label_search).sort('time_stamp'):

#                 doc_id = doc['_id']

#                 trip_coll.update_one({"_id": doc_id}, {
#                     "$set": {
#                         chnk_num: seq
#                     }
#                 })

{'stop_dist': 4518.457092474731, 'time_stamp': 1480877857.0}
{'stop_dist': 2878.32902510714, 'time_stamp': 1480877857.0}
{'stop_dist': 1613.2050994259498, 'time_stamp': 1480877857.0}
{'stop_dist': 113.98719124602371, 'time_stamp': 1480877857.0}
{'stop_dist': 4505.681883585802, 'time_stamp': 1480886723.0}
{'stop_dist': 2867.600748041398, 'time_stamp': 1480886723.0}
{'stop_dist': 1594.3498826269722, 'time_stamp': 1480886723.0}
{'stop_dist': 135.94698180512128, 'time_stamp': 1480886723.0}
{'stop_dist': 4564.0457054113, 'time_stamp': 1480895153.0}
{'stop_dist': 2916.139910206049, 'time_stamp': 1480895153.0}
{'stop_dist': 1684.981637990959, 'time_stamp': 1480895153.0}
{'stop_dist': 39.924136977401915, 'time_stamp': 1480895153.0}
{'stop_dist': 4514.912929681455, 'time_stamp': 1480903485.0}
{'stop_dist': 2875.367511639423, 'time_stamp': 1480903485.0}
{'stop_dist': 1607.8947935882913, 'time_stamp': 1480903485.0}
{'stop_dist': 120.22852990435145, 'time_stamp': 1480903485.0}
{'stop_dist': 4543.4

{'stop_dist': 63.697717802422645, 'time_stamp': 1480904677.0}
{'stop_dist': 4528.671550575489, 'time_stamp': 1480912753.0}
{'stop_dist': 2886.927238664533, 'time_stamp': 1480912753.0}
{'stop_dist': 1628.4109624002474, 'time_stamp': 1480912753.0}
{'stop_dist': 96.69886939886166, 'time_stamp': 1480912753.0}
{'stop_dist': 4541.805327192392, 'time_stamp': 1480949276.0}
{'stop_dist': 2897.1190445933303, 'time_stamp': 1480949276.0}
{'stop_dist': 1651.5818010898208, 'time_stamp': 1480949276.0}
{'stop_dist': 65.7346905958597, 'time_stamp': 1480949276.0}
{'stop_dist': 4502.362378873322, 'time_stamp': 1480956455.0}
{'stop_dist': 2864.3947800598608, 'time_stamp': 1480956455.0}
{'stop_dist': 1590.907219198805, 'time_stamp': 1480956455.0}
{'stop_dist': 137.6632194545638, 'time_stamp': 1480956455.0}
{'stop_dist': 4500.753251897144, 'time_stamp': 1480964006.0}
{'stop_dist': 2862.7208248656725, 'time_stamp': 1480964006.0}
{'stop_dist': 1589.6456562261785, 'time_stamp': 1480964006.0}
{'stop_dist': 137.

{'stop_dist': 4504.289754491307, 'time_stamp': 1480889997.0}
{'stop_dist': 2867.6457938736935, 'time_stamp': 1480889997.0}
{'stop_dist': 1588.2595809317415, 'time_stamp': 1480889997.0}
{'stop_dist': 149.8679843915472, 'time_stamp': 1480889997.0}
{'stop_dist': 4569.614102179118, 'time_stamp': 1480898841.0}
{'stop_dist': 2920.3282618282933, 'time_stamp': 1480898841.0}
{'stop_dist': 1695.9700337406155, 'time_stamp': 1480898841.0}
{'stop_dist': 34.37158118753606, 'time_stamp': 1480898841.0}
{'stop_dist': 4508.75399490015, 'time_stamp': 1480907037.0}
{'stop_dist': 2870.1810024947717, 'time_stamp': 1480907037.0}
{'stop_dist': 1598.857853703861, 'time_stamp': 1480907037.0}
{'stop_dist': 130.67595461049405, 'time_stamp': 1480907037.0}
{'stop_dist': 4575.045194050917, 'time_stamp': 1480914915.0}
{'stop_dist': 2927.767154876715, 'time_stamp': 1480914915.0}
{'stop_dist': 1692.4395655253834, 'time_stamp': 1480914915.0}
{'stop_dist': 52.95554912466285, 'time_stamp': 1480914915.0}
{'stop_dist': 4510

In [190]:
list(trip_coll.find({'chunk_4': {'$exists': False}}).limit(3))

[{'_id': ObjectId('5aef8eb63ad39e98b301548e'),
  'HEADING': '176.0',
  'LATITUDE': '37.7524',
  'LONGITUDE': '-122.40635',
  'PREDICTABLE': '1',
  'REPORT_TIME': '12/04/2016 10:57:37',
  'REV': '1526',
  'SPEED': '6.111',
  'TRAIN_ASSIGNMENT': '3305',
  'VEHICLE_TAG': '5443',
  'time_stamp': 1480877857.0,
  'trip_end': 1,
  'trip_id_iso': '7253960_2016-12-04_LGK2U'},
 {'_id': ObjectId('5aef8eb63ad39e98b3015544'),
  'HEADING': '175.0',
  'LATITUDE': '37.75261',
  'LONGITUDE': '-122.40639',
  'PREDICTABLE': '1',
  'REPORT_TIME': '12/04/2016 13:25:23',
  'REV': '1526',
  'SPEED': '0.0',
  'TRAIN_ASSIGNMENT': '3305',
  'VEHICLE_TAG': '5443',
  'time_stamp': 1480886723.0,
  'trip_end': 1,
  'trip_id_iso': '7253952_2016-12-04_VC1D0'},
 {'_id': ObjectId('5aef8eb63ad39e98b30155f6'),
  'HEADING': '171.0',
  'LATITUDE': '37.75156',
  'LONGITUDE': '-122.40627',
  'PREDICTABLE': '1',
  'REPORT_TIME': '12/04/2016 15:45:53',
  'REV': '1526',
  'SPEED': '6.111',
  'TRAIN_ASSIGNMENT': '3305',
  'VEHIC